Check to see what the data types are of the columns in ReFED_US_State_Food_Surplus_Detail.csv


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/raw/ReFED_US_State_Food_Surplus_Detail.csv', skiprows = 1, nrows = 1000)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   year                                          1000 non-null   int64  
 1   state                                         1000 non-null   object 
 2   sector                                        1000 non-null   object 
 3   sub_sector                                    1000 non-null   object 
 4   sub_sector_category                           1000 non-null   object 
 5   food_type                                     1000 non-null   object 
 6   food_category                                 1000 non-null   object 
 7   tons_surplus                                  1000 non-null   float64
 8   tons_supply                                   1000 non-null   float64
 9   us_dollars_surplus                            1000 non-null   fl

In [5]:
df = pd.read_csv('../data/raw/ReFED_US_State_Food_Surplus_Summary.csv', skiprows = 1, nrows = 1000)

In [6]:
print(df.columns.tolist())

['year', 'state', 'sector', 'sub_sector', 'sub_sector_category', 'food_type', 'tons_surplus', 'tons_supply', 'us_dollars_surplus', 'tons_waste', 'tons_uneaten', 'tons_inedible_parts', 'tons_not_fit_for_human_consumption', 'tons_donations', 'tons_industrial_uses', 'tons_animal_feed', 'tons_anaerobic_digestion', 'tons_composting', 'tons_not_harvested', 'tons_incineration', 'tons_land_application', 'tons_landfill', 'tons_sewer', 'tons_dumping', 'surplus_upstream_100_year_mtco2e_footprint', 'surplus_downstream_100_year_mtco2e_footprint', 'surplus_total_100_year_mtco2e_footprint', 'surplus_upstream_100_year_mtch4_footprint', 'surplus_downstream_100_year_mtch4_footprint', 'surplus_total_100_year_mtch4_footprint', 'gallons_water_footprint', 'meals_wasted']


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   year                                          1000 non-null   int64  
 1   state                                         1000 non-null   object 
 2   sector                                        1000 non-null   object 
 3   sub_sector                                    1000 non-null   object 
 4   sub_sector_category                           1000 non-null   object 
 5   food_type                                     1000 non-null   object 
 6   tons_surplus                                  1000 non-null   float64
 7   tons_supply                                   1000 non-null   float64
 8   us_dollars_surplus                            1000 non-null   float64
 9   tons_waste                                    1000 non-null   fl

In [32]:
df = pd.read_csv('../data/raw/ReFED_US_Food_Waste_Solutions_Detail.csv', skiprows = 1)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186700 entries, 0 to 186699
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   solution_group                              186700 non-null  object 
 1   solution_priority_action_area               186700 non-null  object 
 2   solution_name                               186700 non-null  object 
 3   sector                                      186700 non-null  object 
 4   sub_sector                                  186700 non-null  object 
 5   sub_sector_category                         186700 non-null  object 
 6   state                                       186700 non-null  object 
 7   food_type                                   186700 non-null  object 
 8   annual_tons_diversion_potential             186700 non-null  float64
 9   annual_100_year_mtco2e_reduction_potential  186700 non-null  float64
 

In [34]:
df.columns

Index(['solution_group', 'solution_priority_action_area', 'solution_name',
       'sector', 'sub_sector', 'sub_sector_category', 'state', 'food_type',
       'annual_tons_diversion_potential',
       'annual_100_year_mtco2e_reduction_potential',
       'annual_100_year_mtch4_reduction_potential',
       'annual_gallons_water_savings_potential',
       'annual_meal_equivalents_diverted', 'jobs_created',
       'annual_us_dollars_cost', 'annual_us_dollars_gross_financial_benefit',
       'annual_us_dollars_net_financial_benefit'],
      dtype='object')

In [10]:
df.head()


,solution_group,solution_priority_action_area,solution_name,sector,sub_sector,sub_sector_category,state,food_type,annual_tons_diversion_potential,annual_100_year_mtco2e_reduction_potential,annual_100_year_mtch4_reduction_potential,annual_gallons_water_savings_potential,annual_meal_equivalents_diverted,jobs_created,annual_us_dollars_cost,annual_us_dollars_gross_financial_benefit,annual_us_dollars_net_financial_benefit
0,Prevention,Enhance Product Distribution,Decreased Transit Time,Farm,Not Applicable,Not Applicable,Alabama,Produce,122.648068,21.061844,0.097223,3.774974e+06,2.044134e+05,0.0,7.186987e+04,3.978317e+05,3.259618e+05
1,Prevention,Enhance Product Distribution,Decreased Transit Time,Farm,Not Applicable,Not Applicable,Alaska,Produce,11.612269,2.288086,0.016943,3.427177e+05,1.935378e+04,0.0,6.804609e+03,3.766654e+04,3.086193e+04
2,Prevention,Enhance Product Distribution,Decreased Transit Time,Farm,Not Applicable,Not Applicable,Arizona,Produce,538.745250,91.797857,0.404789,1.662418e+07,8.979088e+05,0.0,3.156964e+05,1.747520e+06,1.431824e+06
3,Prevention,Enhance Product Distribution,Decreased Transit Time,Farm,Not Applicable,Not Applicable,Arkansas,Produce,111.449996,19.230188,0.091263,3.425387e+06,1.857500e+05,0.0,6.530797e+04,3.615087e+05,2.962007e+05
4,Prevention,Enhance Product Distribution,Decreased Transit Time,Farm,Not Applicable,Not Applicable,California,Produce,12177.757085,2092.918189,9.456154,3.750749e+08,2.029626e+07,0.0,7.135977e+06,3.950081e+07,3.236483e+07


In [29]:
df = pd.read_csv('../data/raw/ReFED_US_Food_Waste_Solutions_Summary.csv', skiprows = 1)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   solution_group                              45 non-null     object 
 1   solution_priority_action_area               45 non-null     object 
 2   solution_name                               45 non-null     object 
 3   annual_tons_diversion_potential             45 non-null     float64
 4   annual_100_year_mtco2e_reduction_potential  45 non-null     float64
 5   annual_100_year_mtch4_reduction_potential   45 non-null     float64
 6   annual_gallons_water_savings_potential      45 non-null     float64
 7   annual_meal_equivalents_diverted            45 non-null     float64
 8   jobs_created                                45 non-null     float64
 9   annual_us_dollars_cost                      45 non-null     float64
 10  annual_us_dollar

In [31]:
df.columns

Index(['solution_group', 'solution_priority_action_area', 'solution_name',
       'annual_tons_diversion_potential',
       'annual_100_year_mtco2e_reduction_potential',
       'annual_100_year_mtch4_reduction_potential',
       'annual_gallons_water_savings_potential',
       'annual_meal_equivalents_diverted', 'jobs_created',
       'annual_us_dollars_cost', 'annual_us_dollars_gross_financial_benefit',
       'annual_us_dollars_net_financial_benefit'],
      dtype='object')

In [35]:
df = pd.read_csv('../data/raw/ReFED_US_Food_Waste_Solutions_Financial_Summary_by_Stakeholder.csv', skiprows = 1)

In [36]:
df.shape

(144, 7)

In [37]:
df.head()

,solution_group,solution_priority_action_area,solution_name,stakeholder,annual_us_dollars_cost,annual_us_dollars_gross_financial_benefit,annual_us_dollars_net_financial_benefit
0,Prevention,Enhance Product Distribution,Decreased Transit Time,Foodservice,2.114867e+07,1.170673e+08,9.591862e+07
1,Prevention,Enhance Product Distribution,Decreased Transit Time,Manufacturers,2.998208e+07,1.659642e+08,1.359821e+08
2,Prevention,Enhance Product Distribution,Decreased Transit Time,Producers,4.233568e+07,2.343468e+08,1.920111e+08
3,Prevention,Enhance Product Distribution,Decreased Transit Time,Retailers,2.282315e+08,1.263363e+09,1.035131e+09
4,Prevention,Enhance Product Distribution,First Expired First Out,Foodservice,6.194179e+06,2.373234e+07,1.753816e+07


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 7 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   solution_group                             144 non-null    object 
 1   solution_priority_action_area              144 non-null    object 
 2   solution_name                              144 non-null    object 
 3   stakeholder                                144 non-null    object 
 4   annual_us_dollars_cost                     144 non-null    float64
 5   annual_us_dollars_gross_financial_benefit  144 non-null    float64
 6   annual_us_dollars_net_financial_benefit    144 non-null    float64
dtypes: float64(3), object(4)
memory usage: 8.0+ KB


In [39]:
df.columns

Index(['solution_group', 'solution_priority_action_area', 'solution_name',
       'stakeholder', 'annual_us_dollars_cost',
       'annual_us_dollars_gross_financial_benefit',
       'annual_us_dollars_net_financial_benefit'],
      dtype='object')

In [ ]:
import great_expectations as gx
import pandas as pd
from pathlib import Path

# Load your CSV
csv_path = Path("../data/raw/ReFED_US_Food_Surplus_Summary.csv").resolve()
df = pd.read_csv(csv_path, skiprows=1)

print("Raw DataFrame columns:", df.columns.tolist())

# Start Great Expectations context
context = gx.get_context(mode="ephemeral")

# This works in 0.18.11 to create a Validator directly from a DataFrame
validator = context.sources.pandas_default.read_dataframe(df)

# Print the actual GE-tracked DataFrame columns
print("GE batch columns:", validator.head().columns.tolist())

# Define expectations
validator.expect_column_values_to_not_be_null("tons_surplus")
validator.expect_column_values_to_be_between("tons_surplus", min_value=0)

# Run validation
results = validator.validate()
print("Success?", results["success"])


In [ ]:
import pandas as pd
import sys
from pathlib import Path

# Adds the parent of "scripts" (project root) to the Python path
sys.path.append(str(Path().resolve().parent))

from scripts.cleaning.constants import get_column_diff, get_column_overlap

# File paths
surplus_summary_path                   = Path("../data/raw/ReFED_US_Food_Surplus_Summary.csv").resolve()
surplus_cause_summary_path             = Path("../data/raw/ReFED_US_Food_Surplus_Cause_Summary.csv").resolve()
surplus_detail_path                    = Path("../data/raw/ReFED_US_Food_Surplus_Detail.csv").resolve()
surplus_state_summary_path             = Path("../data/raw/ReFED_US_State_Food_Surplus_Summary.csv").resolve()
surplus_state_detail_path              = Path("../data/raw/ReFED_US_State_Food_Surplus_Detail.csv").resolve()
solutions_detail_path                  = Path("../data/raw/ReFED_US_Food_Waste_Solutions_Detail.csv").resolve()
solutions_summary_path                 = Path("../data/raw/ReFED_US_Food_Waste_Solutions_Summary.csv").resolve()
solutions_state_summary_path           = Path("../data/raw/ReFED_US_State_Food_Waste_Solutions_Summary.csv").resolve()
solutions_financial_summary_path       = Path("../data/raw/ReFED_US_Food_Waste_Solutions_Financial_Summary_by_Stakeholder.csv").resolve()
solutions_state_financial_summary_path = Path("../data/raw/ReFED_US_State_Food_Waste_Solutions_Financial_Summary_by_Stakeholder.csv").resolve()

# Dataframes
df_surplus_summary                    = pd.read_csv(surplus_summary_path, skiprows=1)
df_surplus_cause_summary              = pd.read_csv(surplus_cause_summary_path, skiprows=1)
df_surplus_detail                     = pd.read_csv(surplus_state_detail_path, skiprows=1)
df_surplus_state_summary              = pd.read_csv(surplus_state_summary_path, skiprows=1)
df_surplus_state_detail               = pd.read_csv(surplus_state_detail_path, skiprows=1)
df_solutions_detail                   = pd.read_csv(solutions_detail_path, skiprows=1)
df_solutions_summary                  = pd.read_csv(solutions_summary_path, skiprows=1)
df_solutions_state_summary            = pd.read_csv(solutions_state_summary_path, skiprows=1)
df_solutions_financial_summary        = pd.read_csv(solutions_financial_summary_path, skiprows=1)
df_solutions_state_financial_summary  = pd.read_csv(solutions_state_financial_summary_path, skiprows=1)



In [ ]:
shared_cols_total = get_column_overlap(
    df_surplus_summary, 
    df_surplus_cause_summary, 
    df_surplus_detail, 
    df_surplus_state_detail, 
    df_surplus_state_summary,
    df_solutions_detail,
    #df_solutions_summary,
    #df_solutions_state_summary,
    #df_solutions_financial_summary,
    #df_solutions_state_financial_summary,
    )

diff_surplus_summary = get_column_diff(df_surplus_summary, df_surplus_cause_summary)
diff_surplus_cause_summary = get_column_diff(df_surplus_cause_summary, df_surplus_summary)

#diff_surplus_summary = get_column_diff(df_surplus_summary, df_surplus_cause_summary)
#diff_surplus_cause_summary = get_column_diff(df_surplus_cause_summary, df_surplus_summary)

print("Shared columns:\n", sorted(shared_cols_total))
print("\nColumns only in surplus_summary:\n", sorted(diff_surplus_summary))
print("\nColumns only in surplus_cause_summary:\n", sorted(diff_surplus_cause_summary))

# I'm thinking that the states are going to share columns, but also that the causes are the outlier
shared_cols_state = get_column_overlap(df_surplus_state_detail, df_surplus_state_summary)
diff_surplus_state_summary = get_column_diff(df_surplus_state_summary, df_surplus_state_detail)
diff_surplus_state_detail = get_column_diff(df_surplus_state_detail, df_surplus_state_summary)
print("Shared state columns:\n", sorted(shared_cols_state))
print("\nColumns only in surplus_states_summary:\n", sorted(diff_surplus_state_summary))
print("\nColumns only in surplus_states_detail:\n", sorted(diff_surplus_state_detail))

# Probably the state and US schema for detail are the same
shared_cols_detail = get_column_overlap(df_surplus_detail, df_surplus_state_detail)
diff_cols_detail = get_column_diff(df_surplus_detail, df_surplus_state_detail)
diff_cols_state_detail = get_column_diff(df_surplus_detail, df_surplus_state_detail)


diff_cols_sln = get_column_diff(df_solutions_state_summary, df_surplus_state_summary)
diff_cols_surplus = get_column_diff(df_surplus_state_summary, df_solutions_state_summary)
print("\nShared columns in *_detail:\n", sorted(shared_cols_detail))
print("\nColumns only in surplus_detail:\n", sorted(diff_cols_detail))
print("\nColumns only in surplus_states_detail:\n", sorted(diff_cols_state_detail))
print("\nColumns only in solution:\n", sorted(diff_cols_sln))
print("\nColumns only in surplus:\n", sorted(diff_cols_surplus))



Shared columns:
 []

Columns only in surplus_summary:
 ['gallons_water_footprint', 'meals_wasted', 'sub_sector_category', 'surplus_downstream_100_year_mtch4_footprint', 'surplus_downstream_100_year_mtco2e_footprint', 'surplus_total_100_year_mtch4_footprint', 'surplus_total_100_year_mtco2e_footprint', 'surplus_upstream_100_year_mtch4_footprint', 'surplus_upstream_100_year_mtco2e_footprint', 'tons_anaerobic_digestion', 'tons_animal_feed', 'tons_composting', 'tons_donations', 'tons_dumping', 'tons_incineration', 'tons_industrial_uses', 'tons_land_application', 'tons_landfill', 'tons_not_harvested', 'tons_sewer', 'tons_supply', 'tons_surplus', 'tons_uneaten', 'tons_waste', 'us_dollars_surplus']

Columns only in surplus_cause_summary:
 ['cause_group', 'cause_name', 'tons_surplus_due_to_cause', 'us_dollars_surplus_due_to_cause']
Shared state columns:
 ['food_type', 'gallons_water_footprint', 'meals_wasted', 'sector', 'state', 'sub_sector', 'sub_sector_category', 'surplus_downstream_100_year_

In [ ]:
# Same with the summary (US vs state), except the states_summary has the state as a column

shared_cols_summary     = get_column_overlap(df_surplus_summary, df_surplus_state_summary)
diff_cols_summary       = get_column_diff(df_surplus_summary, df_surplus_state_summary)
diff_cols_state_summary = get_column_diff(df_surplus_state_summary, df_surplus_summary)
print("\nShared columns in *_summary:\n", sorted(shared_cols_summary))
print("\nColumns only in surplus_summary:\n", sorted(diff_cols_summary))
print("\nColumns only in surplus_states_summary:\n", sorted(diff_cols_state_summary))